# Libraries

In [37]:
import pandas as pd

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer


# Read Dataset

In [38]:
raw_df = pd.read_csv('train-balanced-sarcasm.csv')
raw_df.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


# Data-preprocessing 

## Drop Features

Irrelevant: author, date, created_utc

In [39]:
# Drop NAs
raw_df.dropna(inplace=True)

# Select 100000 rows of sample
filter_df = raw_df.sample(n=100000, random_state=000)

# Drop irrelevant features
filter_df.drop(['author', 'date', 'created_utc'],axis=1)

# Data is balance, do not need oversampling
print(filter_df['label'].value_counts()) 

# Show the data
filter_df.head()

0    50016
1    49984
Name: label, dtype: int64


,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
688452,0,"The title of this article should be: ""How to n...",xNOM,MensRights,1,1,0,2014-11,2014-11-07 11:48:34,You can approach women without being creepy
281079,0,What a wasted opportunity... at least be funny...,Jacked1218,MMA,5,5,0,2016-07,2016-07-05 22:06:19,Nate Diaz Snapchat Hacked
742893,1,But....but... sodium!,Chicup,fatlogic,1,1,0,2015-03,2015-03-09 13:16:40,Canned soups have been hugely helpful for the ...
936006,1,"Yeah, we need more animosity between nations.",Bloodysneeze,worldnews,1,1,0,2013-07,2013-07-09 15:20:36,It really sounds like all the English speaking...
1008752,0,uuugh,name032282,Minecraft,2,2,0,2011-04,2011-04-02 04:50:42,Has anyone held a doggy funeral on their serve...


## Categorical Process

Transform comments into TF-IDF vectors

In [40]:
# Instantiate the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2))

# Fit and transform the processed comments and parent_comment
tfidf_comment = tfidf_vectorizer.fit_transform(filter_df['comment'])
tfidf_parent_comment = tfidf_vectorizer.fit_transform(filter_df['parent_comment'])

# Display the shape of the resulting TF-IDF feature matrix
print(tfidf_comment.shape)
print(tfidf_parent_comment.shape)

(100000, 436490)
(100000, 899057)


We have got a comment TF-IDF matrix, containing 100000 rows and 436490 features
We have got a parent_comment TF-IDF matrix, containing 100000 rows and 899057 features

Transform categorical "subreddit" to dummy

In [41]:
categorical_columns = ['subreddit']
for i in categorical_columns: 
    filter_df = pd.concat([filter_df,pd.get_dummies(filter_df[i],drop_first=True, prefix=i)],axis=1)
    filter_df = filter_df.drop(i,axis=1)
    
filter_df.shape

(100000, 5594)

## Split Dataset

In [42]:
# Y is the response variable
Y = filter_df['label']

# X is the features
X = filter_df.drop('label',axis=1)

# Split the data (Train 0.8, Test 0.2)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=000)